In [ ]:
print("Program starting...")
print("Please be patient, it takes around 20 seconds to initialise this program for the first time.")
%pip install beanie bcrypt nest_asyncio --q

import nest_asyncio
import asyncio
import inspect
import math

loop = asyncio.get_event_loop()
nest_asyncio.apply()

import ipywidgets as widgets
from IPython.display import clear_output, display

from abc import ABC, abstractmethod

import time
from google.colab import userdata

from pydantic import BaseModel, Field
from beanie import init_beanie, Document, Indexed
from motor.motor_asyncio import AsyncIOMotorClient

from datetime import date

import random

import bcrypt

############################## Data Models ##############################

class UserModel(Document):
    name: str
    username: Indexed(str, unique=True)
    hashedPassword: str

    class Settings:
        collection = "UserModel"

class Quiz(BaseModel):
    question: str
    solution: str
    answer: str

class QuestionModel(Document):
    ownerId: str
    title: str
    question: Quiz
    dateStamp: date = Field(default_factory=date.today)
    isCorrect: bool = False
    '''
    Every submission counts just before it is answered correctly.
    The default of `timesOfAnswer` is one as the model will only be inserted if the user submit the answer.
    '''
    timesOfAnswering: int = 1
    class Settings:
        collection = "QuestionModel"


############################## Data Models ##############################

############################## Decorator ##############################

def bind(arg: str=None):
    if callable(arg):
        arg.bind = True
        arg.name = None
        return arg
    def decorator(func):
        func.bind = True
        func.name = arg
        return func
    return decorator
############################## Decorator ##############################

class HybridPairs():
    def __init__(self):
        self.ui = []
        self.dic = {}

    def __getitem__(self, key):
        return self.dic[key]

    def add(self, val, key:str=None, *, ignore:bool=False):
        if not ignore:
            self.ui.append(val)
        if key is not None:
            self.dic[key] = val

    def add_many(self, val):
        self.ui.extend(val)

    def to_display(self):
        return self.ui

    def clear(self):
        self.ui.clear()
        self.dic.clear()

class App():
    def __init__(self):
        # App global data
        self.USER_DATA = None

        # the main container
        self.container = widgets.Output()
        display(self.container)

        self.MainMenu = UIMainMenu(app=self, isHold=False)
        self.Register = UIRegister(app=self, isHold=False)
        self.Login = UILogin(app=self, isHold=False)

        self.Dashboard = UIDashboard(app=self)
        self.QuadraticEquations = UIQuadraticEquations(app=self)

def make_title(text, _layout=None):
    return widgets.HTML(f"<h1 style='color: teal'>{text}</h1>",layout=_layout if _layout is not None else widgets.Layout(
        text_align="center"
    ))

class UIBase(ABC):
    def __init__(self, * ,app, isHold=True):
        self.app = app
        self.ui = HybridPairs()
        if not isHold:
            self.build_ui()
            self.bind_events()
        self.isHold = isHold

    @abstractmethod
    def build_ui(self):
        pass

    def bind_events(self):
        for name in dir(self):
            method = getattr(self, name)
            if getattr(method, "bind", False):
                if getattr(method, "name") is not None:
                    name = getattr(method, "name")
                if name not in self.ui.dic:
                    raise KeyError(f"Method name '{name}' not found in UI dictionary.")
                if inspect.iscoroutinefunction(method):
                    # I have no choice but use run_until_complete() to run async function synchronously. Things go so random if I ever try to create a new task (i.e asyncio.create_task()). Maybe a Colab's bug?
                    self.ui[name].on_click(lambda _,m=method: loop.run_until_complete(m(_)))
                else:
                    self.ui[name].on_click(method)


    def show(self, * , isRebuild=False):
        if isRebuild:
            self.ui.clear()
        if isRebuild or self.isHold:
            self.build_ui()
            self.bind_events()

        self.app.container.clear_output()

        with self.app.container:
            display(widgets.VBox(self.ui.to_display(), layout = widgets.Layout(
                max_width ="100%",
                align_items="center"
            )))

class UIMainMenu(UIBase):
    @bind
    def btn_login(self,_):
        self.app.Login.show()

    @bind
    def btn_register(self,_):
        self.app.Register.show()

    @bind
    def btn_exit(self,_):
        print("exit")

    def build_ui(self):
        self.ui.add(make_title("Main Menu"))

        btn_login = widgets.Button(
            description="Login",
            disabled=False,
            button_style="primary"
        )
        self.ui.add(btn_login, "btn_login")

        btn_register = widgets.Button(
            description="Register",
            disabled=False,
            button_style="primary"
        )
        self.ui.add(btn_register, "btn_register")

        btn_exit = widgets.Button(
            description="Exit",
            disabled=False,
            button_style="danger"
        )
        self.ui.add(btn_exit, "btn_exit")

class UIRegister(UIBase):
    def build_ui(self):
        self.ui.add(make_title("Register Page"))

        name = widgets.Text(
            placeholder='Enter your name',
            description='Name: ',
            disabled=False,
            style={'description_width': '140px'}
        )
        self.ui.add(name, "name")

        username = widgets.Text(
            placeholder='Enter your username',
            description='Username: ',
            disabled=False,
            style={'description_width': '140px'}
        )
        self.ui.add(username, "username")

        password = widgets.Password(
            description='Password:',
            disabled=False,
            style={'description_width': '140px'}
        )
        self.ui.add(password, "password")

        confirmed_password = widgets.Password(
            description='Confirmed Password:',
            disabled=False,
            style={'description_width': '140px'}
        )
        self.ui.add(confirmed_password, "confirmed_password")

        error_text_password_not_match = widgets.HTML(
            value="<strong style='color:red'>Password does not match!</strong>",
            layout=widgets.Layout(display="none")
        )
        self.ui.add(error_text_password_not_match, "error_text_password_not_match")

        error_text_password_length = widgets.HTML(
            value="<strong style='color:red'>Password is too short!</strong>",
            layout=widgets.Layout(display="none")
        )
        self.ui.add(error_text_password_length, "error_text_password_length")

        error_text_username = widgets.HTML(
            value="<strong style='color:red'>This username is chosen!</strong>",
            layout=widgets.Layout(display="none")
        )
        self.ui.add(error_text_username, "error_text_username")

        btn_exit = widgets.Button(
            description="Exit",
            disabled=False,
            button_style="danger"
        )
        self.ui.add(btn_exit, "btn_exit", ignore=True)

        btn_register = widgets.Button(
            description="Register",
            disabled=False,
            button_style="primary"
        )
        self.ui.add(btn_register, "btn_register", ignore=True)

        box = widgets.HBox([btn_exit, btn_register], layout=widgets.Layout(
            justify_content="space-between",
            grid_gap="1.5em"
        ))
        self.ui.add(box)

        succeeded = widgets.HTML(
            value="<strong style='color:green'>Successfully registered an account! Click 'exit' to return back.</strong>",
            layout=widgets.Layout(display="none")
        )
        self.ui.add(succeeded, "succeeded")

    @bind
    async def btn_register(self, _):
        from pymongo.errors import DuplicateKeyError
        try:
            btn = self.ui["btn_register"]
            btn.disabled = True

            self.ui["error_text_password_not_match"].layout.display = "none"
            self.ui["error_text_password_length"].layout.display = "none"
            self.ui["error_text_username"].layout.display = "none"
            self.ui["succeeded"].layout.display = "none"

            pwd = self.ui["password"].value
            conf = self.ui["confirmed_password"].value

            # validation
            if pwd != conf:
                self.ui["error_text_password_not_match"].layout.display = ""
                btn.disabled = False
                return

            if len(pwd) < 8:
                self.ui["error_text_password_length"].layout.display = ""
                btn.disabled = False
                return

            salt = bcrypt.gensalt()
            hashed = bcrypt.hashpw(pwd.encode("utf-8"), salt)

            data = UserModel(
                name=self.ui["name"].value,
                username=self.ui["username"].value,
                hashedPassword=hashed
            )

            await data.insert()
            self.ui["succeeded"].layout.display = ""
        except DuplicateKeyError:
            self.ui["error_text_username"].layout.display = ""
        except Exception as e:
            print(e)
        finally:
            btn.disabled = False


    @bind
    def btn_exit(self,_):
        self.app.MainMenu.show()

class UILogin(UIBase):
    def build_ui(self):
        title = make_title("Login Page")
        self.ui.add(title)

        username = widgets.Text(
            placeholder='Enter your username',
            description='Username: ',
            disabled=False,
        )
        self.ui.add(username, "username")

        password = widgets.Password(
            description='Password:',
            disabled=False,
        )
        self.ui.add(password, "password")

        error_text = widgets.HTML(
            value="<strong style='color:red'>Invalid username or/and password</strong>",
            layout=widgets.Layout(display="none")
        )
        self.ui.add(error_text, "error_text")

        btn_exit = widgets.Button(
            description="Exit",
            disabled=False,
            button_style="danger"
        )
        self.ui.add(btn_exit, "btn_exit", ignore=True)

        btn_login = widgets.Button(
            description="Login",
            disabled=False,
            button_style="primary"
        )
        self.ui.add(btn_login, "btn_login", ignore=True)

        box = widgets.HBox([btn_exit, btn_login], layout=widgets.Layout(
            justify_content="space-between",
            grid_gap="1.5em"
        ))
        self.ui.add(box)

    @bind
    async def btn_login(self,_):
        self.ui["error_text"].layout.display = "none"

        btn = self.ui["btn_login"]
        username = self.ui["username"].value
        pwd = self.ui["password"].value

        btn.disabled = True
        result = await UserModel.find_one(UserModel.username == username)
        hash = ""
        fakeHash = bcrypt.hashpw(b"invalid", bcrypt.gensalt()) # the register function restricted the lenght of password to be at least eight-character long. Suggested by https://cheatsheetseries.owasp.org/cheatsheets/Authentication_Cheat_Sheet.html#authentication-responses

        if result is not None:
            hash = result.hashedPassword.encode("utf-8")
        else:
            hash = fakeHash

        isAuth = bcrypt.checkpw(pwd.encode("utf-8"), hash) and result is not None
        if isAuth:
            self.app.USER_DATA = result
            self.app.Dashboard.show(isRebuild=True)
        else:
            self.ui["error_text"].layout.display = ""

        btn.disabled = False

    @bind
    def btn_exit(self,_):
        self.app.MainMenu.show()

class UIDashboard(UIBase):
    NAME_OF_QUADRATIC_EQUATION = "Quadratic Equation"

    def build_ui(self):
        title = make_title("Dashboard")

        btn_sign_out = widgets.Button(
            description="Sign Out",
            disabled=False,
            button_style="danger"
        )
        self.ui.add(btn_sign_out, "btn_sign_out", ignore=True)

        header = widgets.HBox([title, btn_sign_out], layout=widgets.Layout(
            justify_content="space-between",
            align_items="center",
            width="auto",
        ))

        welcome_msg = widgets.HTML(f"<strong>Welcome back! How are you, <span style='color:green'>{self.app.USER_DATA.name}</span>?<strong/>")

        options = widgets.Select(
            options=[self.NAME_OF_QUADRATIC_EQUATION],
            description="Topics: ",
            disabled=False,
            style={
                'description_width': 'initial'
            }
        )
        self.ui.add(options, "options", ignore=True)

        btn_proceed = widgets.Button(
            description="Proceed",
            disabled=False,
            button_style="info"
        )
        self.ui.add(btn_proceed, "btn_proceed", ignore=True)

        container_options = widgets.VBox([options, btn_proceed], layout=widgets.Layout(
            align_items="center",
            width="30em"
        ))

        center = widgets.HBox([welcome_msg, container_options], layout=widgets.Layout(
            margin="1px 0",
            width="auto",
            display="grid"
        ))

        Layout = widgets.AppLayout(
            header=header,
            left_sidebar=None,
            center=center,
            right_sidebar=None,
            footer=None,
            layout=widgets.Layout(
                width="100%",
                padding="1em",
            )
        )

        self.ui.add(Layout)

    @bind
    def btn_sign_out(self,_):
        self.app.MainMenu.show()

    @bind
    def btn_proceed(self,_):
        match self.ui["options"]:
            case NAME_OF_QUADRATIC_EQUATION:
                self.app.QuadraticEquations.show(isRebuild=True)

class UIQuadraticEquations(UIBase):
    def build_ui(self):
        title = make_title("Quadratic Equation Quiz")
        self.ui.add(title)

        instruction = widgets.HTML("<strong>Solve for x for each question</strong>")
        self.ui.add(instruction)

        exit_btn = widgets.Button(
            description="Exit",
            disabled=False,
            button_style="danger"
        )
        self.ui.add(exit_btn, "exit_btn")

        self.ui.add_many(UIQuizHelper(app=self.app,
                                      quiz=self.quiz_generator,
                                      title="Quadratic Equation"
                                      ).build_ui())

    def quiz_generator(self):
        while True:
          a = random.randint(1, 100)
          b = random.randint(-10, 10)
          c = random.randint(-10, 10)
          discriminant = b * b - 4 * a * c
          if discriminant >= 0:
              break

        root_one = (-b + math.sqrt(discriminant)) / (2 * a)
        root_two = (-b - math.sqrt(discriminant)) / (2 * a)

        x = -b / (2 * a)
        Min_Max_Point = a * x * x + b * x + c

        question = "Solve the quadratic equation where a is " + str(a) + ", b is " + str(b) + ", and c is " + str(c)
        solution = "Use the formula: -b ± square root of b × b - 4 × a × c, all divided by 2 × a.\nThen substitute the values of a, b and c to find the roots. Also write the equation in factored and vertex form."
        answer = (
            "The roots are " + str(round(root_one, 3)) + " and " + str(round(root_two, 3)) + ". "
            "\nFactored form is a × (x - root one) × (x - root two). "
            "\nVertex form is a × (x - h)² + k. "
            "\nThe value of x at the minimum or maximum point is " + str(round(x, 3)) +
            ", and the value at that point is " + str(round(Min_Max_Point, 3)) + " (3 Significant Figures or 3 s.f)."
        )

        return Quiz(
            question = question,
            solution = solution,
            answer = answer
        )

    @bind
    def exit_btn(self,_):
        app.Dashboard.show(isRebuild=True)

class UIQuizHelper:
    '''
    the 'queestion' argument must be a calleable function which returns the quiz class
    '''
    def __init__(self, *, app, quiz:Quiz, title:str, numberOfQuestions = 5):
        self.quiz=quiz
        self.title=title
        self.numberOfQuestions=numberOfQuestions
        self.app=app

    def build_ui(self):
        for x in range(self.numberOfQuestions):
            quiz_obj = self.quiz()

            ask = widgets.HTML(f"<strong>{quiz_obj.question}</strong>")

            answer = widgets.Text(
                placeholder="Enter your answer",
                description=f"Question {x+1}:",
                disabled=False
            )

            correct = widgets.HTML("<strong style='color:green'>Correct!</strong>", layout=widgets.Layout(
                display="none"
            ))

            incorrect = widgets.HTML("<strong style='color:red'>Incorrect! Try again or show the solution</strong>", layout=widgets.Layout(
                display="none"
            ))


            solution = widgets.HTML(f"<strong>{quiz_obj.solution}</strong>", layout=widgets.Layout(
                display="none"
            ))

            show_solution_btn = widgets.Button(
                description="Show Solution",
                disabled=False,
                button_style="warning",
                layout=widgets.Layout(
                    display="none"
                )
            )
            def show_solution(_, w_solution = solution):
                w_solution.layout.display=""
            show_solution_btn.on_click(show_solution)

            submit_btn = widgets.Button(
                description="Submit",
                disabled=False,
                button_style="info"
            )
            # Creation of variable after instantiation
            submit_btn.isSubmitted = False
            submit_btn.questionModel = QuestionModel(
                ownerId = str(app.USER_DATA.id),
                title = self.title,
                question = quiz_obj,
                isCorrect = False
            )
            async def submit(
                *,
                myself,
                w_quiz_obj,
                w_answer,
                w_correct,
                w_incorrect,
                w_show_solution_btn
            ):
                myself.disabled = True
                isCorrect = w_answer.value.strip() == w_quiz_obj.answer
                w_correct.layout.display="none"
                w_incorrect.layout.display="none"
                w_show_solution_btn.layout.display="none"
                if isCorrect:
                    w_correct.layout.display=""
                else:
                    w_incorrect.layout.display=""
                    w_show_solution_btn.layout.display=""

                if not myself.isSubmitted:
                    myself.questionModel.isCorrect = isCorrect
                    await myself.questionModel.insert()
                else:
                    prev = myself.questionModel.isCorrect
                    myself.questionModel.isCorrect = isCorrect or myself.questionModel.isCorrect
                    myself.questionModel.timesOfAnswering += 1 if not prev else 0
                    await myself.questionModel.save()
                myself.isSubmitted = True
                myself.disabled = False

            submit_btn.on_click(
                lambda myself,
                w_quiz_obj=quiz_obj,
                w_answer=answer,
                w_incorrect=incorrect,
                w_correct=correct,
                w_show_solution_btn=show_solution_btn:
            loop.run_until_complete(
                submit(
                    myself=myself,
                    w_quiz_obj=w_quiz_obj,
                    w_answer=w_answer,
                    w_correct=w_correct,
                    w_incorrect=w_incorrect,
                    w_show_solution_btn=w_show_solution_btn,
                    )
                )
            )

            btn_container = widgets.HBox([submit_btn, show_solution_btn])
            quiz_container = widgets.VBox([ask, answer, solution, btn_container, correct, incorrect], layout=widgets.Layout(
                border="3px solid",
                padding="1em",
                margin="1em 0"
            ))
            yield quiz_container


############################## ENTRY POINT ##############################
print("Trying to connect to the database...")

# Create a new client and connect to the server
client = AsyncIOMotorClient(userdata.get("MongoDBAtlasConnectionString")) # The connection string will be revoked after WA3 is graded

await init_beanie(database=client["WA3"], document_models=[UserModel, QuestionModel])
await client.admin.command('ping')

print("Connected!")
time.sleep(0.3)

clear_output()
app = App()
app.MainMenu.show()